In [8]:
"""
IEOR 240 Project: Comparing Optimization Methods for ML Training

Methods:
- Gradient Descent (full-batch)
- Stochastic Gradient Descent (mini-batch)
- Nesterov Accelerated Gradient (NAG)
- Adam
- SPSA (Simultaneous Perturbation Stochastic Approximation, gradient-free)

Models:
- Logistic Regression
- Small 2-layer Neural Network

Dataset:
- Breast Cancer dataset from sklearn (binary classification, small & fast)


"""

import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# ---------------------------
# 1. Data loading utilities
# ---------------------------

def load_breast_cancer_torch(test_size=0.2, random_state=0, device="cpu"):
    data = load_breast_cancer()
    X = data.data
    y = data.target  # 0/1

    # Standardize features for stability
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    # Convert to torch tensors
    X_train_t = torch.tensor(X_train, dtype=torch.float32, device=device)
    y_train_t = torch.tensor(y_train, dtype=torch.long, device=device)
    X_test_t = torch.tensor(X_test, dtype=torch.float32, device=device)
    y_test_t = torch.tensor(y_test, dtype=torch.long, device=device)

    return X_train_t, X_test_t, y_train_t, y_test_t


# ---------------------------
# 2. Model definitions
# ---------------------------

class LogisticModel(nn.Module):
    """Logistic regression: linear layer + sigmoid (internally via BCEWithLogitsLoss)."""
    def __init__(self, input_dim):
        super().__init__()
        self.linear = nn.Linear(input_dim, 1)

    def forward(self, x):
        # Return logits (no sigmoid here, handled by loss)
        return self.linear(x).squeeze(-1)


class SmallMLP(nn.Module):
    """Small 2-layer neural network for binary classification."""
    def __init__(self, input_dim, hidden_dim=32):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.act = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, 1)  # binary logits

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x).squeeze(-1)
        return x  # logits


# ---------------------------
# 3. Common training utilities
# ---------------------------

def binary_accuracy_from_logits(logits, y_true):
    """
    logits: tensor of shape (N,)
    y_true: long tensor of {0,1} with shape (N,)
    """
    probs = torch.sigmoid(logits)
    y_pred = (probs >= 0.5).long()
    acc = (y_pred == y_true).float().mean().item()
    return acc


def train_one_epoch(model, optimizer, loss_fn, dataloader, device="cpu"):
    model.train()
    total_loss = 0.0
    n_samples = 0

    for xb, yb in dataloader:
        xb = xb.to(device)
        yb = yb.to(device)

        optimizer.zero_grad()
        logits = model(xb)
        loss = loss_fn(logits, yb.float())  # BCEWithLogitsLoss expects float targets
        loss.backward()
        optimizer.step()

        batch_size = xb.size(0)
        total_loss += loss.item() * batch_size
        n_samples += batch_size

    return total_loss / n_samples


def evaluate_model(model, loss_fn, X, y, device="cpu"):
    model.eval()
    with torch.no_grad():
        logits = model(X.to(device))
        loss = loss_fn(logits, y.to(device).float()).item()
        acc = binary_accuracy_from_logits(logits, y.to(device))
    return loss, acc


# ---------------------------
# 4. SPSA for Logistic Regression
# ---------------------------

def spsa_logistic_train(
    X_train,
    y_train,
    X_val,
    y_val,
    max_iters=200,
    a=0.1,
    c=0.1,
    alpha=0.602,
    gamma=0.101,
    A=10.0,
    device="cpu"
):
    """
    SPSA training for logistic regression:
        theta includes weights and bias as a single vector.
    Uses full-batch loss for simplicity (n is small).

    Returns:
        theta  : final parameter vector (weights + bias)
        history: dict with loss & accuracy over iterations.
    """
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    X_val = X_val.to(device)
    y_val = y_val.to(device)

    n, d = X_train.shape
    # Initialize parameters near zero
    theta = torch.zeros(d + 1, dtype=torch.float32, device=device)

    def loss_fn(theta, X, y):
        w = theta[:-1]
        b = theta[-1]
        logits = X @ w + b
        probs = torch.sigmoid(logits)
        eps = 1e-8
        loss = - (y * torch.log(probs + eps) +
                  (1 - y) * torch.log(1 - probs + eps)).mean()
        return loss

    def acc_fn(theta, X, y):
        w = theta[:-1]
        b = theta[-1]
        logits = X @ w + b
        probs = torch.sigmoid(logits)
        y_pred = (probs >= 0.5).long()
        return (y_pred == y).float().mean().item()

    history = {"iter": [], "train_loss": [], "val_loss": [], "val_acc": []}

    for k in range(max_iters):
        ak = a / ((k + 1 + A) ** alpha)
        ck = c / ((k + 1) ** gamma)

        # Rademacher perturbation: +/- 1
        delta = torch.randint(low=0, high=2, size=theta.shape, device=device) * 2 - 1
        theta_plus = theta + ck * delta
        theta_minus = theta - ck * delta

        # Evaluate loss at perturbed points
        J_plus = loss_fn(theta_plus, X_train, y_train)
        J_minus = loss_fn(theta_minus, X_train, y_train)

        # Estimate gradient
        g_hat = (J_plus - J_minus) / (2.0 * ck) * delta  # element-wise

        # Update
        theta = theta - ak * g_hat

        # Logging
        if (k + 1) % 10 == 0 or k == 0:
            train_loss = loss_fn(theta, X_train, y_train).item()
            val_loss = loss_fn(theta, X_val, y_val).item()
            val_acc = acc_fn(theta, X_val, y_val)

            history["iter"].append(k + 1)
            history["train_loss"].append(train_loss)
            history["val_loss"].append(val_loss)
            history["val_acc"].append(val_acc)

            print(
                f"[SPSA] Iter {k+1:4d} | "
                f"Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
            )

    return theta, history


# ---------------------------
# 4b. SPSA for MLP
# ---------------------------

def spsa_mlp_train(
    X_train,
    y_train,
    X_val,
    y_val,
    input_dim,
    hidden_dim=32,
    max_iters=200,
    a=0.05,
    c=0.05,
    alpha=0.602,
    gamma=0.101,
    A=10.0,
    device="cpu"
):
    """
    SPSA training for the SmallMLP model.
    We flatten all parameters into a single vector theta, perturb it,
    compute loss via forward passes, and update theta via SPSA.
    """
    X_train = X_train.to(device)
    y_train = y_train.to(device)
    X_val = X_val.to(device)
    y_val = y_val.to(device)

    model = SmallMLP(input_dim, hidden_dim=hidden_dim).to(device)
    loss_fn = nn.BCEWithLogitsLoss()

    # --- helpers to flatten / unflatten parameters ---

    params = list(model.parameters())
    shapes = [p.shape for p in params]
    sizes = [p.numel() for p in params]
    cum_sizes = np.cumsum([0] + sizes)

    def flatten_params():
        return torch.cat([p.detach().view(-1) for p in params])

    def set_params(theta_vec):
        with torch.no_grad():
            for i, p in enumerate(params):
                start = cum_sizes[i]
                end = cum_sizes[i + 1]
                p.copy_(theta_vec[start:end].view(shapes[i]))

    theta = flatten_params().to(device)

    def loss_for_theta(theta_vec, X, y):
        set_params(theta_vec)
        with torch.no_grad():
            logits = model(X)
            loss = loss_fn(logits, y.float())
        return loss

    def acc_for_theta(theta_vec, X, y):
        set_params(theta_vec)
        with torch.no_grad():
            logits = model(X)
            probs = torch.sigmoid(logits)
            y_pred = (probs >= 0.5).long()
            acc = (y_pred == y).float().mean().item()
        return acc

    history = {"iter": [], "train_loss": [], "val_loss": [], "val_acc": []}

    for k in range(max_iters):
        ak = a / ((k + 1 + A) ** alpha)
        ck = c / ((k + 1) ** gamma)

        delta = torch.randint(low=0, high=2, size=theta.shape, device=device) * 2 - 1
        theta_plus = theta + ck * delta
        theta_minus = theta - ck * delta

        J_plus = loss_for_theta(theta_plus, X_train, y_train)
        J_minus = loss_for_theta(theta_minus, X_train, y_train)

        g_hat = (J_plus - J_minus) / (2.0 * ck) * delta
        theta = theta - ak * g_hat

        if (k + 1) % 10 == 0 or k == 0:
            train_loss = loss_for_theta(theta, X_train, y_train).item()
            val_loss = loss_for_theta(theta, X_val, y_val).item()
            val_acc = acc_for_theta(theta, X_val, y_val)
            history["iter"].append(k + 1)
            history["train_loss"].append(train_loss)
            history["val_loss"].append(val_loss)
            history["val_acc"].append(val_acc)
            print(
                f"[MLP SPSA] Iter {k+1:4d} | "
                f"Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
            )

    # set final params into model before returning
    set_params(theta)
    return model, history


# ---------------------------
# 5. Experiment runners
# ---------------------------

def run_logistic_experiments(device="cpu"):
    print("=== Logistic Regression Experiments ===")
    X_train, X_test, y_train, y_test = load_breast_cancer_torch(device=device)

    # Train/val split on training set (e.g. 80/20)
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train.cpu().numpy(),
        y_train.cpu().numpy(),
        test_size=0.2,
        random_state=0,
        stratify=y_train.cpu().numpy()
    )
    X_tr = torch.tensor(X_tr, dtype=torch.float32, device=device)
    X_val = torch.tensor(X_val, dtype=torch.float32, device=device)
    y_tr = torch.tensor(y_tr, dtype=torch.long, device=device)
    y_val = torch.tensor(y_val, dtype=torch.long, device=device)

    input_dim = X_tr.shape[1]
    batch_size_sgd = 32
    n_epochs = 50

    # Loss function
    loss_fn = nn.BCEWithLogitsLoss()

    # DataLoaders
    full_train_ds = TensorDataset(X_tr, y_tr)
    full_train_loader = DataLoader(full_train_ds, batch_size=len(full_train_ds), shuffle=True)
    sgd_train_loader = DataLoader(full_train_ds, batch_size=batch_size_sgd, shuffle=True)

    # Store results
    results = {}

    # ---- Gradient Descent (full batch) ----
    model_gd = LogisticModel(input_dim).to(device)
    optimizer_gd = torch.optim.SGD(model_gd.parameters(), lr=0.1)
    print("\n[Logistic] Training with Gradient Descent (full batch)...")
    start = time.time()
    for epoch in range(1, n_epochs + 1):
        train_loss = train_one_epoch(model_gd, optimizer_gd, loss_fn, full_train_loader, device)
        val_loss, val_acc = evaluate_model(model_gd, loss_fn, X_val, y_val, device)
        if epoch % 10 == 0 or epoch == 1:
            print(
                f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
            )
    elapsed_gd = time.time() - start
    test_loss, test_acc = evaluate_model(model_gd, loss_fn, X_test, y_test, device)
    results["GD"] = (test_loss, test_acc, elapsed_gd)

    # ---- SGD (mini-batch) ----
    model_sgd = LogisticModel(input_dim).to(device)
    optimizer_sgd = torch.optim.SGD(model_sgd.parameters(), lr=0.05)
    print("\n[Logistic] Training with SGD (mini-batch)...")
    start = time.time()
    for epoch in range(1, n_epochs + 1):
        train_loss = train_one_epoch(model_sgd, optimizer_sgd, loss_fn, sgd_train_loader, device)
        val_loss, val_acc = evaluate_model(model_sgd, loss_fn, X_val, y_val, device)
        if epoch % 10 == 0 or epoch == 1:
            print(
                f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
            )
    elapsed_sgd = time.time() - start
    test_loss, test_acc = evaluate_model(model_sgd, loss_fn, X_test, y_test, device)
    results["SGD"] = (test_loss, test_acc, elapsed_sgd)

    # ---- Nesterov Accelerated Gradient (NAG) ----
    model_nag = LogisticModel(input_dim).to(device)
    optimizer_nag = torch.optim.SGD(model_nag.parameters(), lr=0.05, momentum=0.9, nesterov=True)
    print("\n[Logistic] Training with Nesterov Accelerated Gradient...")
    start = time.time()
    for epoch in range(1, n_epochs + 1):
        train_loss = train_one_epoch(model_nag, optimizer_nag, loss_fn, sgd_train_loader, device)
        val_loss, val_acc = evaluate_model(model_nag, loss_fn, X_val, y_val, device)
        if epoch % 10 == 0 or epoch == 1:
            print(
                f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
            )
    elapsed_nag = time.time() - start
    test_loss, test_acc = evaluate_model(model_nag, loss_fn, X_test, y_test, device)
    results["NAG"] = (test_loss, test_acc, elapsed_nag)

    # ---- Adam ----
    model_adam = LogisticModel(input_dim).to(device)
    optimizer_adam = torch.optim.Adam(model_adam.parameters(), lr=0.01)
    print("\n[Logistic] Training with Adam...")
    start = time.time()
    for epoch in range(1, n_epochs + 1):
        train_loss = train_one_epoch(model_adam, optimizer_adam, loss_fn, sgd_train_loader, device)
        val_loss, val_acc = evaluate_model(model_adam, loss_fn, X_val, y_val, device)
        if epoch % 10 == 0 or epoch == 1:
            print(
                f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
            )
    elapsed_adam = time.time() - start
    test_loss, test_acc = evaluate_model(model_adam, loss_fn, X_test, y_test, device)
    results["Adam"] = (test_loss, test_acc, elapsed_adam)

    # ---- SPSA ----
    print("\n[Logistic] Training with SPSA (gradient-free)...")
    start = time.time()
    theta_spsa, spsa_hist = spsa_logistic_train(
        X_tr, y_tr, X_val, y_val,
        max_iters=200,
        a=0.1,
        c=0.1,
        alpha=0.602,
        gamma=0.101,
        A=10.0,
        device=device
    )
    elapsed_spsa = time.time() - start

    def spsa_loss_acc(theta, X, y):
        X = X.to(device)
        y = y.to(device)
        w = theta[:-1]
        b = theta[-1]
        logits = X @ w + b
        probs = torch.sigmoid(logits)
        eps = 1e-8
        loss = - (y * torch.log(probs + eps) +
                  (1 - y) * torch.log(1 - probs + eps)).mean().item()
        y_pred = (probs >= 0.5).long()
        acc = (y_pred == y).float().mean().item()
        return loss, acc

    test_loss_spsa, test_acc_spsa = spsa_loss_acc(theta_spsa, X_test, y_test)
    results["SPSA"] = (test_loss_spsa, test_acc_spsa, elapsed_spsa)
    print(f"\n[SPSA] Finished in {elapsed_spsa:.3f} seconds.")

    # Print summary table
    print("\n=== Logistic Regression Summary (Test Set) ===")
    print(f"{'Method':8s} | {'Test Loss':>10s} | {'Test Acc':>9s} | {'Time (s)':>8s}")
    print("-" * 45)
    for method, (loss, acc, t) in results.items():
        print(f"{method:8s} | {loss:10.4f} | {acc:9.4f} | {t:8.3f}")


def run_mlp_experiments(device="cpu"):
    print("\n\n=== Small Neural Network Experiments ===")
    X_train, X_test, y_train, y_test = load_breast_cancer_torch(device=device)

    # Train/val split on training set
    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train.cpu().numpy(),
        y_train.cpu().numpy(),
        test_size=0.2,
        random_state=0,
        stratify=y_train.cpu().numpy()
    )
    X_tr = torch.tensor(X_tr, dtype=torch.float32, device=device)
    X_val = torch.tensor(X_val, dtype=torch.float32, device=device)
    y_tr = torch.tensor(y_tr, dtype=torch.long, device=device)
    y_val = torch.tensor(y_val, dtype=torch.long, device=device)

    input_dim = X_tr.shape[1]
    batch_size = 32
    n_epochs = 20  # smaller, NN is more expensive than logistic

    loss_fn = nn.BCEWithLogitsLoss()

    train_ds = TensorDataset(X_tr, y_tr)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

    results = {}

    # ---- GD (full batch) ----
    model_gd = SmallMLP(input_dim, hidden_dim=32).to(device)
    optimizer_gd = torch.optim.SGD(model_gd.parameters(), lr=0.1)
    full_train_loader = DataLoader(train_ds, batch_size=len(train_ds), shuffle=True)

    print("\n[MLP] Training with Gradient Descent (full batch)...")
    start = time.time()
    for epoch in range(1, n_epochs + 1):
        train_loss = train_one_epoch(model_gd, optimizer_gd, loss_fn, full_train_loader, device)
        val_loss, val_acc = evaluate_model(model_gd, loss_fn, X_val, y_val, device)
        if epoch % 5 == 0 or epoch == 1:
            print(
                f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
            )
    elapsed_gd = time.time() - start
    test_loss, test_acc = evaluate_model(model_gd, loss_fn, X_test, y_test, device)
    results["GD"] = (test_loss, test_acc, elapsed_gd)

    # ---- SGD ----
    model_sgd = SmallMLP(input_dim, hidden_dim=32).to(device)
    optimizer_sgd = torch.optim.SGD(model_sgd.parameters(), lr=0.05)
    print("\n[MLP] Training with SGD (mini-batch)...")
    start = time.time()
    for epoch in range(1, n_epochs + 1):
        train_loss = train_one_epoch(model_sgd, optimizer_sgd, loss_fn, train_loader, device)
        val_loss, val_acc = evaluate_model(model_sgd, loss_fn, X_val, y_val, device)
        if epoch % 5 == 0 or epoch == 1:
            print(
                f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
            )
    elapsed_sgd = time.time() - start
    test_loss, test_acc = evaluate_model(model_sgd, loss_fn, X_test, y_test, device)
    results["SGD"] = (test_loss, test_acc, elapsed_sgd)

    # ---- NAG ----
    model_nag = SmallMLP(input_dim, hidden_dim=32).to(device)
    optimizer_nag = torch.optim.SGD(model_nag.parameters(), lr=0.05, momentum=0.9, nesterov=True)
    print("\n[MLP] Training with Nesterov Accelerated Gradient...")
    start = time.time()
    for epoch in range(1, n_epochs + 1):
        train_loss = train_one_epoch(model_nag, optimizer_nag, loss_fn, train_loader, device)
        val_loss, val_acc = evaluate_model(model_nag, loss_fn, X_val, y_val, device)
        if epoch % 5 == 0 or epoch == 1:
            print(
                f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
            )
    elapsed_nag = time.time() - start
    test_loss, test_acc = evaluate_model(model_nag, loss_fn, X_test, y_test, device)
    results["NAG"] = (test_loss, test_acc, elapsed_nag)

    # ---- Adam ----
    model_adam = SmallMLP(input_dim, hidden_dim=32).to(device)
    optimizer_adam = torch.optim.Adam(model_adam.parameters(), lr=0.01)
    print("\n[MLP] Training with Adam...")
    start = time.time()
    for epoch in range(1, n_epochs + 1):
        train_loss = train_one_epoch(model_adam, optimizer_adam, loss_fn, train_loader, device)
        val_loss, val_acc = evaluate_model(model_adam, loss_fn, X_val, y_val, device)
        if epoch % 5 == 0 or epoch == 1:
            print(
                f"Epoch {epoch:3d} | Train Loss: {train_loss:.4f} | "
                f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}"
            )
    elapsed_adam = time.time() - start
    test_loss, test_acc = evaluate_model(model_adam, loss_fn, X_test, y_test, device)
    results["Adam"] = (test_loss, test_acc, elapsed_adam)

    # ---- SPSA for MLP ----
    print("\n[MLP] Training with SPSA (gradient-free)...")
    start = time.time()
    model_spsa, spsa_hist_mlp = spsa_mlp_train(
        X_tr, y_tr, X_val, y_val,
        input_dim=input_dim,
        hidden_dim=32,
        max_iters=200,
        a=0.05,
        c=0.05,
        alpha=0.602,
        gamma=0.101,
        A=10.0,
        device=device
    )
    elapsed_spsa_mlp = time.time() - start
    test_loss_spsa, test_acc_spsa = evaluate_model(model_spsa, loss_fn, X_test, y_test, device)
    results["SPSA"] = (test_loss_spsa, test_acc_spsa, elapsed_spsa_mlp)

    # Summary table
    print("\n=== MLP Summary (Test Set) ===")
    print(f"{'Method':8s} | {'Test Loss':>10s} | {'Test Acc':>9s} | {'Time (s)':>8s}")
    print("-" * 45)
    for method, (loss, acc, t) in results.items():
        print(f"{method:8s} | {loss:10.4f} | {acc:9.4f} | {t:8.3f}")


# ---------------------------
# 6. Main
# ---------------------------

if __name__ == "__main__":
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    run_logistic_experiments(device=device)
    run_mlp_experiments(device=device)


Using device: cuda
=== Logistic Regression Experiments ===

[Logistic] Training with Gradient Descent (full batch)...
Epoch   1 | Train Loss: 0.7955 | Val Loss: 0.5887 | Val Acc: 0.7912
Epoch  10 | Train Loss: 0.2577 | Val Loss: 0.2587 | Val Acc: 0.9231
Epoch  20 | Train Loss: 0.1918 | Val Loss: 0.1928 | Val Acc: 0.9451
Epoch  30 | Train Loss: 0.1627 | Val Loss: 0.1615 | Val Acc: 0.9560
Epoch  40 | Train Loss: 0.1455 | Val Loss: 0.1422 | Val Acc: 0.9560
Epoch  50 | Train Loss: 0.1340 | Val Loss: 0.1288 | Val Acc: 0.9780

[Logistic] Training with SGD (mini-batch)...
Epoch   1 | Train Loss: 0.4473 | Val Loss: 0.3260 | Val Acc: 0.9451
Epoch  10 | Train Loss: 0.1284 | Val Loss: 0.1196 | Val Acc: 0.9780
Epoch  20 | Train Loss: 0.1014 | Val Loss: 0.0902 | Val Acc: 0.9670
Epoch  30 | Train Loss: 0.0903 | Val Loss: 0.0766 | Val Acc: 0.9670
Epoch  40 | Train Loss: 0.0837 | Val Loss: 0.0690 | Val Acc: 0.9890
Epoch  50 | Train Loss: 0.0791 | Val Loss: 0.0633 | Val Acc: 0.9890

[Logistic] Training